In [14]:
import pandas as pd
import numpy as np
import time as t
from sklearn.utils import shuffle
from sklearn.preprocessing import LabelEncoder
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import Perceptron, LogisticRegression
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import classification_report, accuracy_score
from tabulate import tabulate
import warnings
warnings.filterwarnings('ignore')

# Load data
data = pd.read_csv("AI-Data.csv")

# Display table-based analysis
def display_table(title, table):
    print(f"\n📊 {title}")
    print(tabulate(table, headers="keys", tablefmt="fancy_grid"))

# Table menu
def table_menu():
    options = {
        1: ("Marks Class Count", data['Class'].value_counts().rename_axis("Class").reset_index(name="Count")),
        2: ("Semester-wise", pd.crosstab(data['Semester'], data['Class'])),
        3: ("Gender-wise", pd.crosstab(data['gender'], data['Class'])),
        4: ("Nationality-wise", pd.crosstab(data['NationalITy'], data['Class'])),
        5: ("Grade-wise", pd.crosstab(data['GradeID'], data['Class'])),
        6: ("Section-wise", pd.crosstab(data['SectionID'], data['Class'])),
        7: ("Topic-wise", pd.crosstab(data['Topic'], data['Class'])),
        8: ("Stage-wise", pd.crosstab(data['StageID'], data['Class'])),
        9: ("Absent Days-wise", pd.crosstab(data['StudentAbsenceDays'], data['Class']))
    }

    while True:
        print("\n📌 Select a Table to View:")
        for key, (title, _) in options.items():
            print(f"{key}. {title}")
        print("10. Continue to ML Models\n")
        ch = int(input("Enter choice: "))
        if ch in options:
            display_table(options[ch][0], options[ch][1])
            input("\nPress Enter to continue...")
        elif ch == 10:
            break
        else:
            print("Invalid choice. Try again.")

# Drop and preprocess
data_clean = data.drop(columns=["gender", "StageID", "GradeID", "NationalITy", "PlaceofBirth",
                                "SectionID", "Topic", "Semester", "Relation",
                                "ParentschoolSatisfaction", "ParentAnsweringSurvey", "AnnouncementsView"])

data_clean = shuffle(data_clean)

# Encode categorical
for col in data_clean.columns:
    if data_clean[col].dtype == object:
        data_clean[col] = LabelEncoder().fit_transform(data_clean[col])

# Train/test split
split_idx = int(len(data_clean) * 0.7)
X = data_clean.iloc[:, :-1]
y = data_clean.iloc[:, -1]
X_train, X_test = X.iloc[:split_idx], X.iloc[split_idx:]
y_train, y_test = y.iloc[:split_idx], y.iloc[split_idx:]

# Models
models = {
    "Decision Tree": DecisionTreeClassifier(),
    "Random Forest": RandomForestClassifier(),
    "Perceptron": Perceptron(),
    "Logistic Regression": LogisticRegression(),
    "Neural Network (MLP)": MLPClassifier(activation="logistic")
}

results = []
for name, model in models.items():
    model.fit(X_train, y_train)
    preds = model.predict(X_test)
    acc = accuracy_score(y_test, preds)
    report = classification_report(y_test, preds, output_dict=True)
    results.append({
        "Model": name,
        "Accuracy": round(acc, 3),
        "Precision (avg)": round(report['weighted avg']['precision'], 3),
        "Recall (avg)": round(report['weighted avg']['recall'], 3),
        "F1-score (avg)": round(report['weighted avg']['f1-score'], 3)
    })

# Display model summary
print("\n✅ Model Performance Summary:")
print(tabulate(results, headers="keys", tablefmt="grid"))

# Predict user input
def custom_prediction():
    choice = input("\n🤖 Do you want to test with custom input? (y/n): ")
    if choice.lower() != "y":
        print("Goodbye!")
        return

    try:
        rai = int(input("Raised Hands: "))
        res = int(input("Visited Resources: "))
        dis = int(input("Discussions: "))
        absc = input("Absent Days (Under-7 / Above-7): ")
        absc = 1 if absc == "Under-7" else 0
        inp = np.array([rai, res, dis, absc]).reshape(1, -1)

        print("\n🎯 Model Predictions:")
        for name, model in models.items():
            pred = model.predict(inp)[0]
            label = "H" if pred == 0 else "M" if pred == 1 else "L"
            print(f"{name:25}: {label}")

    except Exception as e:
        print("Error with input. Please try again.\n", str(e))

# Run everything
table_menu()
custom_prediction()



✅ Model Performance Summary:
+----------------------+------------+-------------------+----------------+------------------+
| Model                |   Accuracy |   Precision (avg) |   Recall (avg) |   F1-score (avg) |
+======================+============+===================+================+==================+
| Decision Tree        |      0.681 |             0.689 |          0.681 |            0.679 |
+----------------------+------------+-------------------+----------------+------------------+
| Random Forest        |      0.701 |             0.699 |          0.701 |            0.7   |
+----------------------+------------+-------------------+----------------+------------------+
| Perceptron           |      0.458 |             0.21  |          0.458 |            0.288 |
+----------------------+------------+-------------------+----------------+------------------+
| Logistic Regression  |      0.708 |             0.711 |          0.708 |            0.707 |
+----------------------+------